In [29]:
import requests
from bs4 import BeautifulSoup
import csv
from selenium import webdriver
import time
import re
import pandas as pd
from datetime import datetime

For first link, find the match by clicking on:
https://www.cricket.com/series/4554/indian-premier-league-2024/matches

For second link, find the match by clicking on:
https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/match-schedule-fixtures-and-results


In [99]:
matchid=0 #temp
crcom = f"https://www.cricket.com/live-score/240682/commentary/mi-vs-rcb-eliminator-women-s-premier-league-2024"
espn = f"https://www.espncricinfo.com/series/women-s-premier-league-2023-24-1411373/mumbai-indians-women-vs-royal-challengers-bangalore-women-eliminator-1417736/full-scorecard"

In [97]:
### URL PASTE ZONE ###
input1=input('Paste cricket.com link here:')
crcom=f"{input1}"
input2=input('Paste espncricinfo.com link here:')
espn=f"{input2}"

Paste cricket.com link here:https://www.cricket.com/live-score/237771/match-info/csk-vs-rcb-match-1-indian-premier-league-2024
Paste espncricinfo.com link here:https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/chennai-super-kings-vs-royal-challengers-bangalore-1st-match-1422119/live-cricket-score


In [100]:
matchid=0 #temp
# Set up the WebDriver (you need to have the appropriate WebDriver installed, like chromedriver or geckodriver)
driver = webdriver.Chrome()

# Navigate to the URL
driver.get(crcom)

# Scroll the page to the bottom
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(10)  # Add a delay to let content load
    new_height = driver.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        break

    last_height = new_height

# Get the HTML source after scrolling
html_source = driver.page_source

# Close the WebDriver
driver.quit()

# Parse the HTML using BeautifulSoup
soup = BeautifulSoup(html_source, 'html.parser')

In [5]:
driver2 = webdriver.Chrome()

# Navigate to the URL
driver2.get(espn)

# Scroll the page to the bottom
last_height = driver2.execute_script("return document.body.scrollHeight")

while True:
    driver2.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(10)  # Add a delay to let content load
    new_height = driver2.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        break

    last_height = new_height

# Get the HTML source after scrolling
html_source = driver2.page_source

# Close the WebDriver
driver2.quit()

soup2 = BeautifulSoup(html_source, 'html.parser')

In [101]:
### MATCHID AND DATE ###
#***UNCOMMENT FOR IPL***

# mtchid = soup2.find_all("div", class_="ds-text-tight-xs ds-truncate ds-text-typo-mid3 ds-mb-1")
# mtchidr = [tag.text.strip() for tag in mtchid]
# mtch=mtchidr[0].split(',')
# m=mtch[0]

# pattern = r'\d+'
# matches = re.findall(pattern, m)

# num = int(matches[0])
# if num < 10:
#     num = f"0{num}"
# print("Numeric part:", num)

# print(mtch)
# mtchidf = num
# matchid = int("2024"+str(mtchidf))
# print(matchid)
datf=mtch[2]

default_year = "2024"  # You can change this to any default year you want
date_object = datetime.strptime(datf.strip() + " " + default_year, "%B %d %Y")
date = date_object.strftime("%d-%m-%Y")

print(date)

['Eliminator (N)', ' Delhi', ' March 15', ' 2024', " Women's Premier League"]
15-03-2024


In [32]:
# <p class="lable-light dark:lable text-sm min-w-[25%]">MI</p>
teams = soup.find_all("p", class_=["lable-light dark:lable text-sm min-w-[25%]"])
teamorder = [tag.get_text() for tag in teams]
print(teamorder)

['RCB', 'MI', 'RCB', 'MI']


In [33]:
#Delivery data
d = soup.find_all("p", class_=["font-medium text-black dark:text-white text-xs lg:text-sm pt-1"])
deli = [float(tag.get_text()) for tag in d]
deli.reverse()
print(len(deli))

243


In [34]:
#res data
v = soup.find_all("div", class_="flex flex-col bg-lightbg dark:bg-gray rounded items-center")
r = [tag.find('p').text.strip() for tag in v]
r.reverse()
print(len(r))

243


In [10]:
#Player name data
n = soup.find_all("span", class_= "font-semibold")
names = [tag.get_text() for tag in n]
names.reverse()
names = [element.strip(' ,') for element in names]
print(len(names))
batters = names[::2]  # Items at even indices
bowlers = names[1::2]
print(len(batters))
print(len(bowlers))

486
243
243


In [11]:
#Combining ball-by-ball data
result_list = list(zip(deli, r, batters, bowlers))
print(len(result_list))

243


In [12]:
# # IMPACT PLAYER ###
#***UNCOMMENT FOR IPL***


# ipw = soup2.find_all("div", class_=["ds-text-tight-s ds-font-regular ds-pb-2 last:ds-pb-0"])
# print(ipw)
# impactp = [tag.get_text() for tag in ipw]
# llist=impactp[0]+impactp[1]
# llistl=llist.split("\xa0")
# dd= [x[:-4] for x in llistl if x[-4:]==" in,"]
dd = ['Alyssa Healy','Titas Sadhu']
# print(dd)

In [13]:
### BATTER AND DISMISSAL TYPE###
tbody = soup2.find_all("tbody")
first = None
second = None

for i, tbody in enumerate(tbody, 1):
    p_tags_inside_container = tbody.find_all('tr')
    p_texts_list = []
    
    for p_tag in p_tags_inside_container:
        p_texts_list.append(p_tag.text)
    if i == 1:
        first = p_texts_list
    elif i == 3:
        second = p_texts_list
# print(first)
# print(second)
info=p_texts_list

venue=info[0]
print(venue)

new_list = []
for element in first:
    new_list.append(element)
    if "extras" in element.lower():
        break
new_list=new_list[:-1]

string_to_remove = '\xa0'
trimmed_list = []

for index, element in enumerate(new_list):
    if index % 2 == 0 and string_to_remove in element:
        trimmed_element = element.split(string_to_remove, 1)[0]
        trimmed_list.append(trimmed_element)
    else:
        trimmed_list.append(element)

firstinnwicket = [(trimmed_list[i], trimmed_list[i + 1]) for i in range(0, len(trimmed_list), 2)]
print(firstinnwicket)

new_list = []
for element in second:
    new_list.append(element)
    if "extras" in element.lower():
        break
new_list=new_list[:-1]

string_to_remove = '\xa0'
trimmed_list = []

for index, element in enumerate(new_list):
    if index % 2 == 0 and string_to_remove in element:
        trimmed_element = element.split(string_to_remove, 1)[0]
        trimmed_list.append(trimmed_element)
    else:
        trimmed_list.append(element)

secondinnwicket = [(trimmed_list[i], trimmed_list[i + 1]) for i in range(0, len(trimmed_list), 2)]
print(secondinnwicket)

Arun Jaitley Stadium, Delhi
[('Smriti Mandhana', 'c Ismail b Nat Sciver-Brunt'), ('Sophie Devine', ' b Matthews'), ('Ellyse Perry', 'c Nat Sciver-Brunt b Ishaque'), ('Disha Kasat', 'c Vastrakar b Ishaque'), ('Richa Ghosh', 'c Nat Sciver-Brunt b Matthews'), ('Sophie Molineux', ' b Nat Sciver-Brunt'), ('Georgia Wareham', 'not out '), ('Shreyanka Patil', 'not out ')]
[('Yastika Bhatia', ' b Perry'), ('Hayley Matthews', 'c Wareham b Patil'), ('Nat Sciver-Brunt', ' b Wareham'), ('Harmanpreet Kaur', 'c Devine b Patil'), ('Amelia Kerr', 'not out '), ('Sajeevan Sajana', 'st †Ghosh b Molineux'), ('Pooja Vastrakar', 'st †Ghosh b Sobhana'), ('Amanjot Kaur', 'not out ')]


In [14]:
data = result_list
innings=0
batterdismissed=None
# Define the header for your CSV file
header = ['matchid','date', 'venue' ,'innings', 'over', 'delivery', 'battingteam', 'bowlingteam', 'Column2', 'res', 'impbat', 'batters', 'bowlers', 'impbowl', 'wide', 'extras', 'noball', 'legbye', 'bye', 'batterrun', 'wicket','batterdismissed']
# Specify the name of your CSV file
csv_file = f"{matchid}.csv"

# Write data to CSV file
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    
    # Write the header
    writer.writerow(header)
    
    # Initialize variables to store previous values
    prev_delivery = None
    prev_data = None
    # Write the data
    for tuple_data in data:
        delivery, column2, batters, bowlers = tuple_data
        res = column2
        wide = 1 if 'wd' in column2 else 0
        noball = 1 if 'nb' in column2 else 0
        legbye = 1 if 'lb' in column2 else 0
        bye = 1 if 'b' in column2 and 'lb' not in column2 else 0
        batterrun = int(column2) if column2.isdigit() else 0
        wicket = 1 if 'W' in column2 else 0
        impbat = 1 if batters in dd else 0
        impbowl =1 if bowlers in dd else 0
        if delivery == 0.1:
            innings+=1
        if innings >2:
            innings=2
        if innings == 1:
            battingteam = teamorder[0]
            bowlingteam = teamorder[1]
        elif innings == 2:
            battingteam = teamorder[1]
            bowlingteam = teamorder[0]
        over = int(delivery)+1
            
        if noball == 1:
                delivery = str(delivery) + '*'
        # Calculate the value for the "extras" column
        if 'wd' in column2:
            numeric_part = ''.join(filter(str.isdigit, column2))
            extras = 1 + int(numeric_part) if numeric_part.isdigit() else 1
        elif 'nb' in column2 in column2:
            numeric_part = ''.join(filter(str.isdigit, column2))
            batterrun = int(numeric_part) if numeric_part.isdigit() else 1
        elif 'lb' in column2 or 'b' in column2:
            numeric_part = ''.join(filter(str.isdigit, column2))
            extras = int(numeric_part) if numeric_part.isdigit() else 0
        else:
            extras = 0
        if delivery == prev_delivery:
            # Accumulate data from previous row
            wide += prev_data[14]
            extras += prev_data[15]
            noball += prev_data[16]
            legbye += prev_data[17]
            batterrun += prev_data[18]
            wicket += prev_data[19]
        if 'wd' not in column2:
            writer.writerow([matchid, date, venue, innings, over, delivery, battingteam, bowlingteam, column2, res, impbat, batters, bowlers, impbowl, wide, extras, noball, legbye, bye, batterrun, wicket, batterdismissed])

        # Store current values as previous values for the next iteration
        prev_delivery = delivery
        prev_data = [matchid, date, venue, innings, over, delivery, battingteam, bowlingteam, column2, res, impbat, batters, bowlers, impbowl, wide, extras, noball, legbye, bye, batterrun, wicket, batterdismissed]

print(f'The data has been successfully written to {csv_file}.')

The data has been successfully written to 0.csv.


In [15]:
###BATTER AND DISMISSAL DELIVERY###

split1 = re.split(r',\s*(?![^()]*\))', first[-1])
fow1 = []
for item in split1:
    match = re.search(r'\((.*?)\)', item)
    if match:
        content_within_parentheses = match.group(1)
        name, number_str = map(str.strip, content_within_parentheses.split(', '))
        number = float(number_str[:-3]) if '.' in number_str[:-3] else int(number_str[:-3])
        fow1.append((name, number))
        
split2 = re.split(r',\s*(?![^()]*\))', second[-1])
fow2 = []
for item in split2:
    match = re.search(r'\((.*?)\)', item)
    if match:
        content_within_parentheses = match.group(1)
        name, number_str = map(str.strip, content_within_parentheses.split(', '))
        number = float(number_str[:-3]) if '.' in number_str[:-3] else int(number_str[:-3])
        fow2.append((name, number))
print(fow1)
print(fow2)

[('Sophie Devine', 1.6), ('Smriti Mandhana', 2.2), ('Disha Kasat', 3.4), ('Richa Ghosh', 9.1), ('Sophie Molineux', 14.6), ('Ellyse Perry', 19.2)]
[('Hayley Matthews', 3.5), ('Yastika Bhatia', 7.6), ('Nat Sciver-Brunt', 10.4), ('Harmanpreet Kaur', 17.6), ('Sajeevan Sajana', 18.5), ('Pooja Vastrakar', 19.4)]


In [23]:
df = pd.read_csv(f'{matchid}.csv')

for tuple_item in fow1:
    tuple_batter, tuple_delivery = tuple_item
    
    condition = (df['delivery'] == str(tuple_delivery)) & (df['innings'] == 1) & (df['wicket'] == 1)
    df.loc[condition, ['batterdismissed']] = [tuple_batter]

for tuple_item in fow2:
    tuple_batter, tuple_delivery = tuple_item
    
    condition = (df['delivery'] == str(tuple_delivery)) & (df['innings'] == 2) & (df['wicket'] == 1)
    df.loc[condition, ['batterdismissed']] = [tuple_batter]
df.to_csv(f'{matchid}.csv', index=False)

In [17]:
### DISMISSAL TYPE###
wiclist1=[]
for tuple_item in firstinnwicket:

    tuple_batter, tuple_dismissal = tuple_item
    if 'c ' in tuple_dismissal and 'b ' in tuple_dismissal:
        wic='Caught Out'
    elif 'st ' in tuple_dismissal and 'b ' in tuple_dismissal and 'c ' not in tuple_dismissal:
        wic='Stumped Out'
    elif 'b ' in tuple_dismissal and 'c ' not in tuple_dismissal and 'st ' not in tuple_dismissal:
        wic='Bowled'
    elif 'run out' in tuple_dismissal:
        wic='Run Out'
    elif 'lbw ' in tuple_dismissal:
        wic='LBW'
    elif 'hit wicket' in tuple_dismissal:
        wic='Hit Wicket'
    else:
        wic='Not Out'
    tuple_item = tuple_item + (wic,)
    wiclist1.append(tuple_item)
print(wiclist1)

wiclist2=[]
for tuple_item in secondinnwicket:

    tuple_batter, tuple_dismissal = tuple_item
    if 'c ' in tuple_dismissal and 'b ' in tuple_dismissal:
        wic='Caught Out'
    elif 'st ' in tuple_dismissal and 'b ' in tuple_dismissal and 'c ' not in tuple_dismissal:
        wic='Stumped Out'
    elif 'b ' in tuple_dismissal and 'c ' not in tuple_dismissal and 'st ' not in tuple_dismissal:
        wic='Bowled'
    elif 'run out (' in tuple_dismissal:
        wic='Run Out'
    elif 'lbw ' in tuple_dismissal:
        wic='LBW'
    elif 'hit wicket' in tuple_dismissal:
        wic='Hit Wicket'
    elif 'not out' :
        wic='Not Out'
    tuple_item = tuple_item + (wic,)
    wiclist2.append(tuple_item)
print(wiclist2)

[('Smriti Mandhana', 'c Ismail b Nat Sciver-Brunt', 'Caught Out'), ('Sophie Devine', ' b Matthews', 'Bowled'), ('Ellyse Perry', 'c Nat Sciver-Brunt b Ishaque', 'Caught Out'), ('Disha Kasat', 'c Vastrakar b Ishaque', 'Caught Out'), ('Richa Ghosh', 'c Nat Sciver-Brunt b Matthews', 'Caught Out'), ('Sophie Molineux', ' b Nat Sciver-Brunt', 'Bowled'), ('Georgia Wareham', 'not out ', 'Not Out'), ('Shreyanka Patil', 'not out ', 'Not Out')]
[('Yastika Bhatia', ' b Perry', 'Bowled'), ('Hayley Matthews', 'c Wareham b Patil', 'Caught Out'), ('Nat Sciver-Brunt', ' b Wareham', 'Bowled'), ('Harmanpreet Kaur', 'c Devine b Patil', 'Caught Out'), ('Amelia Kerr', 'not out ', 'Not Out'), ('Sajeevan Sajana', 'st †Ghosh b Molineux', 'Stumped Out'), ('Pooja Vastrakar', 'st †Ghosh b Sobhana', 'Stumped Out'), ('Amanjot Kaur', 'not out ', 'Not Out')]


In [24]:
df = pd.read_csv(f'{matchid}.csv')

for tuple_item in wiclist1:
    tuple_batter, tuple_dismissal, tuple_type = tuple_item
    condition = (df['batterdismissed'] == str(tuple_batter)) & (df['innings'] == 1)
    df.loc[condition, ['dismissaltype']] = [tuple_type]

for tuple_item in wiclist2:
    tuple_batter, tuple_delivery, tuple_type = tuple_item
    condition = (df['batterdismissed'] == str(tuple_batter)) & (df['innings'] == 2)
    df.loc[condition, ['dismissaltype']] = [tuple_type]
    
df.to_csv(f'{matchid}.csv', index=False)

In [35]:
### MATCH FLOW ###
divs_with_teams = soup2.find_all("div", class_=["ds-mb-4"])
matchflow = []

specific_team_soup = BeautifulSoup(str(divs_with_teams[6]), 'html.parser')

ul_tags_for_specific_team = specific_team_soup.find_all("ul", class_=["ds-text-tight-s", "ds-font-regular", "ds-list-disc", "ds-pt-2", "ds-px-4", "ds-mb-4"])
        
current_list_variable = []
for ul_tag in ul_tags_for_specific_team:
    p_texts_list = [p_tag.text.strip() for p_tag in ul_tag.find_all("li")]
    current_list_variable.extend(p_texts_list)

matchflow = current_list_variable

first_innings = []
second_innings = []

current_innings = first_innings

# Iterate through the data
for element in matchflow:
    if "Innings Break" in element:
        current_innings = second_innings
    current_innings.append(element)

print(first_innings)
second_innings = second_innings[1:]
print(second_innings)

timeout1=[]
timeout2=[]
for element in first_innings:
    if 'Strategic Timeout:' in element:
        match = re.search(r'(\d+\.\d+) overs', element)
        if match:
            over_number = float(match.group(1))
            if over_number.is_integer() and over_number > 0:
                over_number -= 1
                over_number += 0.6
            timeout1.append(over_number)
print(timeout1)
for element in second_innings:
    if 'Strategic Timeout:' in element:
        match = re.search(r'(\d+\.\d+) overs', element)
        if match:
            over_number = float(match.group(1))
            if over_number.is_integer() and over_number > 0:
                over_number -= 1
                over_number += 0.6
            timeout2.append(over_number)
print(timeout2)
df['total']=df.batterrun + df.extras
df['score']=df.groupby(['innings']).total.cumsum()
df['wicketfell']=df.groupby(['innings']).wicket.cumsum()
df.to_csv(f'{matchid}.csv', index=False)

['Powerplay 1: Overs 0.1 - 6.0 (Mandatory - 34 runs, 3 wickets)', 'Strategic Timeout: Royal Challengers Bangalore Women - 47/3 in 8.0 overs (EA Perry 12, RM Ghosh 13)', 'Royal Challengers Bangalore Women: 50 runs in 9.5 overs (59 balls),  Extras 2', 'Strategic Timeout: Royal Challengers Bangalore Women - 90/5 in 16.0 overs (EA Perry 40, G Wareham 2)', 'Royal Challengers Bangalore Women: 100 runs in 16.4 overs (100 balls),  Extras 3', 'EA Perry: 50 off 40 balls (5 x 4, 1 x 6)']
['Over 0.1: Review by Royal Challengers Bangalore Women (Bowling), Decision Challenged - Wicket, Umpire - Gayathri V, Batter - YH Bhatia (Struck down)', 'Powerplay 1: Overs 0.1 - 6.0 (Mandatory - 37 runs, 1 wicket)', 'Strategic Timeout: Mumbai Indians Women - 37/1 in 6.0 overs (YH Bhatia 16, Nat Sciver-Brunt 1)', 'Mumbai Indians Women: 50 runs in 7.5 overs (47 balls),  Extras 5', 'Over 9.2: Review by Mumbai Indians Women (Batting), Decision Challenged - Wide, Umpire - P Joshi, Batter - H Kaur (Upheld)', 'Strategi

In [20]:
df = pd.read_csv(f'{matchid}.csv')
for i in timeout1:
    condition = (df['delivery'] == str(i)) & (df['innings'] == 1)
    df.loc[condition, ['strategictimeout']] = 1
    df.loc[~condition, ['strategictimeout']] = 0
for i in timeout2:
    condition = (df['delivery'] == str(i)) & (df['innings'] == 2)
    df.loc[condition, ['strategictimeout']] = 1
    df.loc[~condition, ['strategictimeout']] = 0
df.to_csv(f'{matchid}.csv', index=False)

In [21]:
df =pd.read_csv(f'{matchid}.csv')
df['battingposition'] = None

# Iterating over rows of DataFrame and list of tuples
for index, row in df.iterrows():
    if row['innings'] == 1:  # Check if innings is equal to 1
        for i, (player, _, _) in enumerate(wiclist1):
            if i<=10:
                if player == row['batters']:
                    df.at[index, 'battingposition'] = i+1
                    break

# Displaying the DataFrame with the new column


# Iterating over rows of DataFrame and list of tuples
for index, row in df.iterrows():
    if row['innings'] == 2:  # Check if innings is equal to 1
        for i, (player, _, _) in enumerate(wiclist2):
            if i<=10:
                if player == row['batters']:
                    df.at[index, 'battingposition'] = i+1
                    break
df['bowlerwicket'] = None
condition= (df['dismissaltype'] != 'Run Out') & (df['wicket'] == 1)
df.loc[condition,['bowlerwicket']] = 1
# Displaying the DataFrame with the new column
print(df)
df.to_csv(f'{matchid}.csv', index= False)

     matchid        date                        venue  innings  over delivery  \
0          0  15-03-2024  Arun Jaitley Stadium, Delhi        1     1      0.1   
1          0  15-03-2024  Arun Jaitley Stadium, Delhi        1     1      0.2   
2          0  15-03-2024  Arun Jaitley Stadium, Delhi        1     1      0.3   
3          0  15-03-2024  Arun Jaitley Stadium, Delhi        1     1      0.4   
4          0  15-03-2024  Arun Jaitley Stadium, Delhi        1     1      0.5   
..       ...         ...                          ...      ...   ...      ...   
236        0  15-03-2024  Arun Jaitley Stadium, Delhi        2    20     19.2   
237        0  15-03-2024  Arun Jaitley Stadium, Delhi        2    20     19.3   
238        0  15-03-2024  Arun Jaitley Stadium, Delhi        2    20     19.4   
239        0  15-03-2024  Arun Jaitley Stadium, Delhi        2    20     19.5   
240        0  15-03-2024  Arun Jaitley Stadium, Delhi        2    20     19.6   

    battingteam bowlingteam

In [102]:
df.columns

Index(['matchid', 'date', 'venue', 'innings', 'over', 'delivery',
       'battingteam', 'bowlingteam', 'Column2', 'res', 'impbat', 'batters',
       'bowlers', 'impbowl', 'wide', 'extras', 'noball', 'legbye', 'bye',
       'batterrun', 'wicket', 'batterdismissed', 'dismissaltype', 'total',
       'score', 'wicketfell', 'strategictimeout', 'battingposition',
       'bowlerwicket', 'powerplay'],
      dtype='object')

In [90]:
index1= []

# Iterate through the list and check for 'Powerplay'
for index, item in enumerate(first_innings):
    if 'Powerplay' in item:
        index1.append(index)

print(index1)

pp1=first_innings[index1[0]].strip()
print(pp1)
substring1 = re.search(r'Overs\s(.*?)\s\(', pp1).group(1)
split_result1 = substring1.split(' - ')

print(split_result1)
start_value1 = split_result1[0]
end_value1 = split_result1[1]

start_value1 = float(start_value1)
end_value1 = float(end_value1)

if start_value1.is_integer() and start_value1 > 0:
    start_value1 -= 0.4
if end_value1.is_integer() and end_value1 > 0:
    end_value1 -= 0.4
start_value1=int(start_value1)+1
end_value1=int(end_value1)+1

index2= []

# Iterate through the list and check for 'Powerplay'
for index, item in enumerate(second_innings):
    if 'Powerplay' in item:
        index2.append(index)

print(index2)            
pp2=second_innings[index2[0]].strip()
print(pp2)
substring2 = re.search(r'Overs\s(.*?)\s\(', pp2).group(1)
split_result2 = substring1.split(' - ')

print(split_result2)
start_value2 = split_result2[0]
end_value2 = split_result2[1]

start_value2 = float(start_value2)
end_value2 = float(end_value2)

if start_value2.is_integer() and start_value2 > 0:
    start_value2 -= 0.4
if end_value2.is_integer() and end_value2 > 0:
    end_value2 -= 0.4
    
start_value2=int(start_value2)+1
end_value2=int(end_value2)+1

[0]
Powerplay 1: Overs 0.1 - 6.0 (Mandatory - 34 runs, 3 wickets)
['0.1', '6.0']
[1]
Powerplay 1: Overs 0.1 - 6.0 (Mandatory - 37 runs, 1 wicket)
['0.1', '6.0']


In [92]:
df=pd.read_csv(f'{matchid}.csv')
# mask1 = (df['innings'==1])& (df['over'] >= start_value1) & (df['over'] <= end_value1)
# mask = df['over'].between(1, 6)
# # Set the values in the 'pp' column to 1 where the mask is True
df['powerplay'] = 0

# Set 'pp' column to 1 where innings is 1 and over falls between start and end (inclusive)
df.loc[(df['innings'] == 1) & (df['over'].between(start_value1, end_value1, inclusive='both')), 'powerplay'] = 1
df.loc[(df['innings'] == 2) & (df['over'].between(start_value2, end_value2, inclusive='both')), 'powerplay'] = 1

# Display the DataFrame
df.to_csv(f'{matchid}.csv', index= False)